# Label Taxonomy & Data Enrichment EDA

**Goal**: Survey all four datasets' metadata, examine label distributions, visualize sample images
from each category, and evaluate classification taxonomy options for our triage system.

**Key questions**:
1. What are the original label distributions across all datasets?
2. How do current binary mappings (benign/malignant) hold up clinically?
3. Should we include a "normal skin / no lesion" class for real-world triage?
4. What augmentation strategies are most relevant given our domain gaps?
5. Do we need additional datasets?

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from PIL import Image
import warnings
warnings.filterwarnings('ignore')

sns.set_theme(style='whitegrid', font_scale=1.1)
DATA_DIR = Path('../data')

print('Data directory contents:')
for p in sorted(DATA_DIR.iterdir()):
    print(f'  {p.name}/')

## 1. Load All Metadata (Without Images)

We load CSVs only -- no PIL Image loading -- so this runs fast.

In [ ]:
# --- HAM10000 ---
ham_df = pd.read_csv(DATA_DIR / 'HAM10000_metadata.csv')
ham_df['dataset'] = 'HAM10000'
ham_df['domain'] = 'dermoscopic'
print(f'HAM10000: {len(ham_df)} rows')
print(f'  Columns: {list(ham_df.columns)}')
print(f'  dx values: {ham_df["dx"].value_counts().to_dict()}')
print()

In [ ]:
# --- DDI ---
ddi_df = pd.read_csv(DATA_DIR / 'ddi' / 'ddi_metadata.csv')
ddi_df['dataset'] = 'DDI'
ddi_df['domain'] = 'clinical'
print(f'DDI: {len(ddi_df)} rows')
print(f'  Columns: {list(ddi_df.columns)}')
print(f'  malignant distribution: {ddi_df["malignant"].value_counts().to_dict()}')
print(f'  skin_tone distribution: {ddi_df["skin_tone"].value_counts().to_dict()}')
print(f'  disease values ({ddi_df["disease"].nunique()} unique): {ddi_df["disease"].value_counts().head(10).to_dict()}')
print()

In [ ]:
# --- Fitzpatrick17k ---
fitz_df = pd.read_csv(DATA_DIR / 'fitzpatrick17k' / 'fitzpatrick17k.csv')
fitz_df['dataset'] = 'Fitzpatrick17k'
fitz_df['domain'] = 'clinical'
print(f'Fitzpatrick17k: {len(fitz_df)} rows')
print(f'  Columns: {list(fitz_df.columns)}')
print(f'  three_partition_label: {fitz_df["three_partition_label"].value_counts().to_dict()}')
print(f'  fitzpatrick distribution: {fitz_df["fitzpatrick_scale"].value_counts().sort_index().to_dict()}')
print(f'  Unique conditions (label): {fitz_df["label"].nunique()}')
print()

In [ ]:
# --- PAD-UFES-20 ---
pad_df = pd.read_csv(DATA_DIR / 'pad_ufes' / 'metadata.csv')
pad_df['dataset'] = 'PAD-UFES-20'
pad_df['domain'] = 'smartphone'
print(f'PAD-UFES-20: {len(pad_df)} rows')
print(f'  Columns: {list(pad_df.columns)}')
print(f'  diagnostic: {pad_df["diagnostic"].value_counts().to_dict()}')
print(f'  fitspatrick distribution: {pad_df["fitspatrick"].value_counts().sort_index().to_dict()}')
print(f'  fitspatrick missing: {pad_df["fitspatrick"].isna().sum()}')
print()

## 2. Original Label Distributions

Visualize the raw label distributions across all four datasets before any binary mapping.

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# HAM10000 - original 7-class
ham_counts = ham_df['dx'].value_counts()
colors_ham = ['#e74c3c' if dx in ('mel','bcc','akiec') else '#2ecc71' for dx in ham_counts.index]
ham_counts.plot.barh(ax=axes[0,0], color=colors_ham)
axes[0,0].set_title('HAM10000 — Original 7 Classes\n(red=malignant, green=benign)')
axes[0,0].set_xlabel('Count')

# DDI - disease breakdown colored by malignancy
ddi_disease = ddi_df.groupby(['disease','malignant']).size().reset_index(name='count')
ddi_disease = ddi_disease.sort_values('count', ascending=True).tail(15)
colors_ddi = ['#e74c3c' if m else '#2ecc71' for m in ddi_disease['malignant']]
axes[0,1].barh(ddi_disease['disease'], ddi_disease['count'], color=colors_ddi)
axes[0,1].set_title('DDI — Top 15 Diseases\n(red=malignant, green=benign)')
axes[0,1].set_xlabel('Count')

# Fitzpatrick17k - three_partition_label
fitz_counts = fitz_df['three_partition_label'].value_counts()
color_map = {'benign': '#2ecc71', 'malignant': '#e74c3c', 'non-neoplastic': '#3498db'}
fitz_colors = [color_map.get(x, '#95a5a6') for x in fitz_counts.index]
fitz_counts.plot.bar(ax=axes[1,0], color=fitz_colors)
axes[1,0].set_title('Fitzpatrick17k — Three-Partition Labels')
axes[1,0].set_ylabel('Count')
axes[1,0].tick_params(axis='x', rotation=0)

# PAD-UFES-20 - diagnostic categories
pad_counts = pad_df['diagnostic'].value_counts()
colors_pad = ['#e74c3c' if dx in ('BCC','MEL','SCC','ACK') else '#2ecc71' for dx in pad_counts.index]
pad_counts.plot.bar(ax=axes[1,1], color=colors_pad)
axes[1,1].set_title('PAD-UFES-20 — Diagnostic Categories\n(red=malignant/precancerous, green=benign)')
axes[1,1].set_ylabel('Count')
axes[1,1].tick_params(axis='x', rotation=0)

plt.tight_layout()
plt.savefig('../results/label_distributions_raw.png', dpi=150, bbox_inches='tight')
plt.show()

## 3. Current Binary Mapping Analysis

Our current code maps everything to binary (0=benign, 1=malignant). Let's examine
what gets grouped together and whether this makes clinical sense.

In [ ]:
# Current binary mappings from the codebase
print('=== Current Binary Mappings ===')
print()
print('HAM10000 (BINARY_MAPPING in loader.py):')
ham_binary = {
    'akiec': ('MALIGNANT', 'Actinic Keratoses — pre-cancerous'),
    'bcc':   ('MALIGNANT', 'Basal Cell Carcinoma — slow-growing cancer'),
    'mel':   ('MALIGNANT', 'Melanoma — aggressive cancer'),
    'bkl':   ('BENIGN',    'Benign Keratosis — seborrheic keratosis, solar lentigo'),
    'df':    ('BENIGN',    'Dermatofibroma — benign fibrous nodule'),
    'nv':    ('BENIGN',    'Melanocytic Nevi — common moles'),
    'vasc':  ('BENIGN',    'Vascular Lesions — angiomas, pyogenic granulomas'),
}
for dx, (label, desc) in ham_binary.items():
    count = (ham_df['dx'] == dx).sum()
    print(f'  {dx:8s} -> {label:10s}  (n={count:5d})  {desc}')

print()
print('PAD-UFES-20 (PAD_BINARY_MAPPING in pad_ufes.py):')
pad_binary = {
    'ACK': ('MALIGNANT', 'Actinic Keratosis — pre-cancerous'),
    'BCC': ('MALIGNANT', 'Basal Cell Carcinoma'),
    'MEL': ('MALIGNANT', 'Melanoma'),
    'SCC': ('MALIGNANT', 'Squamous Cell Carcinoma'),
    'NEV': ('BENIGN',    'Melanocytic Nevus — common moles'),
    'SEK': ('BENIGN',    'Seborrheic Keratosis'),
}
for dx, (label, desc) in pad_binary.items():
    count = (pad_df['diagnostic'] == dx).sum()
    print(f'  {dx:8s} -> {label:10s}  (n={count:5d})  {desc}')

print()
print('Fitzpatrick17k (three_partition_label):')
print('  benign         -> BENIGN     (includes: eczema, acne, warts, etc.)')
print('  malignant      -> MALIGNANT  (includes: melanoma, BCC, SCC, etc.)')
print('  non-neoplastic -> EXCLUDED   (inflammatory, infectious conditions)')
for part in ['benign', 'malignant', 'non-neoplastic']:
    count = (fitz_df['three_partition_label'] == part).sum()
    print(f'    {part:20s} n={count}')

print()
print('DDI:')
print(f'  malignant=True  -> MALIGNANT  n={(ddi_df["malignant"]==True).sum()}')
print(f'  malignant=False -> BENIGN     n={(ddi_df["malignant"]==False).sum()}')

## 4. Clinical Taxonomy Considerations

### Problem: "Benign" is overloaded

Our current binary maps several clinically distinct categories into "benign":

| What we call "benign" | Clinical reality | Triage action |
|---|---|---|
| **Melanocytic nevi** (moles) | Normal, no action needed | "All clear" |
| **Seborrheic keratosis** | Cosmetic concern, age-related | "Monitor" |
| **Dermatofibroma** | Benign tumor, may want removal | "Monitor" |
| **Vascular lesions** | Usually benign but variable | "Monitor" |
| **Non-neoplastic** (Fitz17k) | Eczema, psoriasis, infections | Needs treatment! |

### Problem: Missing "normal skin" class

All four datasets contain **only lesion images** pre-selected by clinicians.
A real-world triage app will receive:
- Photos of normal skin with no lesion
- Scars, tattoos, insect bites
- Non-skin images entirely

Without a "normal / not a lesion" class, the model MUST classify every input
as either benign or malignant, leading to high false positive rates.

In [ ]:
# Quantify the binary class balance across all datasets
print('=== Binary Class Balance After Current Mapping ===')
print()

# HAM10000
ham_binary_map = {'akiec':1, 'bcc':1, 'bkl':0, 'df':0, 'mel':1, 'nv':0, 'vasc':0}
ham_labels = ham_df['dx'].map(ham_binary_map)
print(f'HAM10000:       benign={sum(ham_labels==0):5d}  malignant={sum(ham_labels==1):5d}  '
      f'ratio={sum(ham_labels==0)/sum(ham_labels==1):.1f}:1')

# DDI
ddi_labels = ddi_df['malignant'].astype(int)
print(f'DDI:            benign={sum(ddi_labels==0):5d}  malignant={sum(ddi_labels==1):5d}  '
      f'ratio={sum(ddi_labels==0)/sum(ddi_labels==1):.1f}:1')

# Fitzpatrick17k (excluding non-neoplastic)
fitz_binary = fitz_df[fitz_df['three_partition_label'].isin(['benign','malignant'])]
fitz_labels = (fitz_binary['three_partition_label'] == 'malignant').astype(int)
print(f'Fitzpatrick17k: benign={sum(fitz_labels==0):5d}  malignant={sum(fitz_labels==1):5d}  '
      f'ratio={sum(fitz_labels==0)/max(sum(fitz_labels==1),1):.1f}:1')

# PAD-UFES-20
pad_binary_map = {'ACK':1, 'BCC':1, 'MEL':1, 'SCC':1, 'NEV':0, 'SEK':0}
pad_labels = pad_df['diagnostic'].map(pad_binary_map)
print(f'PAD-UFES-20:    benign={sum(pad_labels==0):5d}  malignant={sum(pad_labels==1):5d}  '
      f'ratio={sum(pad_labels==0)/sum(pad_labels==1):.1f}:1')

# Also show what happens to Fitzpatrick17k non-neoplastic
n_nonneo = (fitz_df['three_partition_label'] == 'non-neoplastic').sum()
print(f'\nFitzpatrick17k non-neoplastic (currently EXCLUDED): {n_nonneo}')
print('  These are inflammatory/infectious conditions that DO need medical attention')
print('  but are NOT cancerous. Excluding them loses ~{:.0f}% of Fitz17k data.'.format(
    100 * n_nonneo / len(fitz_df)))

## 5. Sample Image Survey

Visual survey of representative images from each dataset and category.
This helps us understand domain differences and what the model sees.

In [ ]:
def show_samples(img_paths, titles, suptitle, ncols=5, figsize=(20, 4)):
    """Display a grid of sample images."""
    n = min(len(img_paths), ncols)
    fig, axes = plt.subplots(1, n, figsize=figsize)
    if n == 1:
        axes = [axes]
    for ax, path, title in zip(axes, img_paths[:n], titles[:n]):
        try:
            img = Image.open(path)
            ax.imshow(img)
            ax.set_title(title, fontsize=9)
        except Exception as e:
            ax.text(0.5, 0.5, f'Error: {e}', ha='center', va='center', transform=ax.transAxes)
        ax.axis('off')
    fig.suptitle(suptitle, fontsize=13, fontweight='bold')
    plt.tight_layout()
    plt.show()

# HAM10000 - one per class
ham_img_dir = DATA_DIR / 'Skin Cancer' / 'Skin Cancer'
for dx_class in ['mel', 'bcc', 'akiec', 'nv', 'bkl', 'df', 'vasc']:
    subset = ham_df[ham_df['dx'] == dx_class].sample(n=min(5, (ham_df['dx']==dx_class).sum()), random_state=42)
    paths = [ham_img_dir / f'{iid}.jpg' for iid in subset['image_id']]
    titles = [f'{dx_class}\n{row.get("localization","")}' for _, row in subset.iterrows()]
    show_samples(paths, titles, f'HAM10000: {dx_class} (domain=dermoscopic)')

In [ ]:
# DDI - samples by skin tone and malignancy
ddi_img_dir = DATA_DIR / 'ddi' / 'images'
for tone in [12, 34, 56]:
    for mal in [True, False]:
        subset = ddi_df[(ddi_df['skin_tone']==tone) & (ddi_df['malignant']==mal)]
        subset = subset.sample(n=min(5, len(subset)), random_state=42)
        paths = [ddi_img_dir / f for f in subset['DDI_file']]
        titles = [f'{row["disease"]}' for _, row in subset.iterrows()]
        tone_label = {12:'I-II (light)', 34:'III-IV (medium)', 56:'V-VI (dark)'}[tone]
        mal_label = 'malignant' if mal else 'benign'
        show_samples(paths, titles, f'DDI: skin_tone={tone_label}, {mal_label}')

In [ ]:
# Fitzpatrick17k - samples by three_partition_label
fitz_img_dir = DATA_DIR / 'fitzpatrick17k' / 'images'

for partition in ['benign', 'malignant', 'non-neoplastic']:
    subset = fitz_df[fitz_df['three_partition_label'] == partition].sample(
        n=min(5, (fitz_df['three_partition_label']==partition).sum()), random_state=42)
    # Try to find images (may not all be downloaded yet)
    paths = []
    titles = []
    for _, row in subset.iterrows():
        md5 = row['md5hash']
        for ext in ['.jpg', '.png', '.jpeg']:
            p = fitz_img_dir / f'{md5}{ext}'
            if p.exists():
                paths.append(p)
                titles.append(f'{row["label"][:30]}\nFST={row.get("fitzpatrick_scale","?")}')
                break
    if paths:
        show_samples(paths, titles, f'Fitzpatrick17k: {partition} (domain=clinical)')
    else:
        print(f'No images found for Fitzpatrick17k {partition} (download may be in progress)')

In [ ]:
# PAD-UFES-20 - samples by diagnostic category
pad_img_dir = DATA_DIR / 'pad_ufes' / 'images'

for dx in ['MEL', 'BCC', 'SCC', 'ACK', 'NEV', 'SEK']:
    subset = pad_df[pad_df['diagnostic'] == dx].sample(
        n=min(5, (pad_df['diagnostic']==dx).sum()), random_state=42)
    paths = [pad_img_dir / row['img_id'] for _, row in subset.iterrows()]
    titles = [f'{dx}\nFST={row.get("fitspatrick","?")} age={row.get("age","?")}' 
              for _, row in subset.iterrows()]
    show_samples(paths, titles, f'PAD-UFES-20: {dx} (domain=smartphone)')

## 6. Fitzpatrick Skin Type Coverage

Critical for fairness analysis: how well do our datasets cover the Fitzpatrick spectrum?

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# DDI (grouped)
ddi_tone = ddi_df['skin_tone'].value_counts().sort_index()
ddi_tone.index = ['I-II', 'III-IV', 'V-VI']
ddi_tone.plot.bar(ax=axes[0], color=['#f5cba7', '#d4a574', '#6b4226'])
axes[0].set_title('DDI — Skin Tone (Grouped FST)')
axes[0].set_ylabel('Count')
axes[0].tick_params(axis='x', rotation=0)

# Fitzpatrick17k
fitz_fst = fitz_df['fitzpatrick_scale'].value_counts().sort_index()
fitz_colors = ['#fde8d0', '#f5cba7', '#d4a574', '#a67b5b', '#6b4226', '#3d2314']
fitz_fst.plot.bar(ax=axes[1], color=fitz_colors[:len(fitz_fst)])
axes[1].set_title('Fitzpatrick17k — FST Distribution')
axes[1].set_ylabel('Count')
axes[1].tick_params(axis='x', rotation=0)

# PAD-UFES-20
pad_fst = pad_df['fitspatrick'].dropna().astype(int).value_counts().sort_index()
pad_colors = ['#fde8d0', '#f5cba7', '#d4a574', '#a67b5b', '#6b4226', '#3d2314']
pad_fst.plot.bar(ax=axes[2], color=pad_colors[:len(pad_fst)])
axes[2].set_title('PAD-UFES-20 — FST Distribution')
axes[2].set_ylabel('Count')
axes[2].tick_params(axis='x', rotation=0)

plt.suptitle('Fitzpatrick Skin Type Coverage Across Datasets', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.savefig('../results/fitzpatrick_coverage.png', dpi=150, bbox_inches='tight')
plt.show()

print('\nNote: HAM10000 has NO Fitzpatrick annotations.')
print('DDI is the ONLY dataset with deliberate balance across skin tones.')
print(f'PAD-UFES-20 has {pad_df["fitspatrick"].isna().sum()} missing FST values ({100*pad_df["fitspatrick"].isna().mean():.0f}%)')

## 7. Domain & Imaging Condition Survey

Compare image properties across domains to understand the domain gap.

In [ ]:
def get_image_stats(img_path, max_dim=256):
    """Get basic image statistics."""
    img = Image.open(img_path).convert('RGB')
    w, h = img.size
    # Resize for faster computation
    img_small = img.resize((min(w, max_dim), min(h, max_dim)))
    arr = np.array(img_small, dtype=np.float32)
    return {
        'width': w, 'height': h, 'aspect_ratio': w/h,
        'mean_r': arr[:,:,0].mean(), 'mean_g': arr[:,:,1].mean(), 'mean_b': arr[:,:,2].mean(),
        'std': arr.std(), 'brightness': arr.mean(),
    }

# Sample images from each dataset
np.random.seed(42)
stats_rows = []

# HAM10000 - sample 100
for iid in ham_df.sample(100, random_state=42)['image_id']:
    p = ham_img_dir / f'{iid}.jpg'
    if p.exists():
        s = get_image_stats(p)
        s['dataset'] = 'HAM10000'
        s['domain'] = 'dermoscopic'
        stats_rows.append(s)

# DDI - all 656
for f in ddi_df.sample(min(100, len(ddi_df)), random_state=42)['DDI_file']:
    p = ddi_img_dir / f
    if p.exists():
        s = get_image_stats(p)
        s['dataset'] = 'DDI'
        s['domain'] = 'clinical'
        stats_rows.append(s)

# PAD-UFES-20 - sample 100
for iid in pad_df.sample(100, random_state=42)['img_id']:
    p = pad_img_dir / iid
    if p.exists():
        s = get_image_stats(p)
        s['dataset'] = 'PAD-UFES-20'
        s['domain'] = 'smartphone'
        stats_rows.append(s)

# Fitzpatrick17k - sample 100 (if downloaded)
fitz_available = list(fitz_img_dir.glob('*.*'))[:100]
for p in fitz_available:
    try:
        s = get_image_stats(p)
        s['dataset'] = 'Fitz17k'
        s['domain'] = 'clinical'
        stats_rows.append(s)
    except: pass

stats_df = pd.DataFrame(stats_rows)
print(f'Computed stats for {len(stats_df)} images')
stats_df.groupby('dataset')[['brightness', 'std', 'width', 'height', 'aspect_ratio']].describe().round(1)

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# Brightness distribution by domain
for domain in stats_df['dataset'].unique():
    subset = stats_df[stats_df['dataset'] == domain]
    axes[0].hist(subset['brightness'], bins=30, alpha=0.5, label=domain, density=True)
axes[0].set_title('Brightness Distribution by Dataset')
axes[0].set_xlabel('Mean Brightness')
axes[0].legend()

# Color channel means (R vs G scatter)
for domain in stats_df['dataset'].unique():
    subset = stats_df[stats_df['dataset'] == domain]
    axes[1].scatter(subset['mean_r'], subset['mean_g'], alpha=0.4, label=domain, s=10)
axes[1].set_title('Color Distribution (R vs G channel)')
axes[1].set_xlabel('Mean Red')
axes[1].set_ylabel('Mean Green')
axes[1].legend()

# Resolution distribution
for domain in stats_df['dataset'].unique():
    subset = stats_df[stats_df['dataset'] == domain]
    total_pixels = subset['width'] * subset['height']
    axes[2].hist(total_pixels / 1e6, bins=30, alpha=0.5, label=domain, density=True)
axes[2].set_title('Resolution Distribution (Megapixels)')
axes[2].set_xlabel('Megapixels')
axes[2].legend()

plt.suptitle('Domain Gap: Imaging Condition Differences', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.savefig('../results/domain_gap_analysis.png', dpi=150, bbox_inches='tight')
plt.show()

## 8. Augmentation Strategy Review

Our codebase has two augmentation pipelines. Let's visualize what they do.

In [ ]:
import sys
sys.path.insert(0, '..')

try:
    from src.data.dermoscope_aug import (
        DermoscopeVignetteAdd, DermoscopeVignetteRemove,
        DermoscopeGelBubbles, DermoscopeColorCast, HairOverlay,
        get_dermoscope_removal_pipeline, get_dermoscope_addition_pipeline,
    )
    HAS_AUGS = True
except Exception as e:
    HAS_AUGS = False
    print(f'Could not import augmentations (albumentations/cv2 issue): {e}')
    print('Skipping augmentation demo — see Section 11 markdown for strategy review.')

if HAS_AUGS:
    # Pick a sample dermoscopic image and a smartphone image
    derm_img = np.array(Image.open(ham_img_dir / (ham_df.iloc[0]['image_id'] + '.jpg')).convert('RGB'))
    phone_img = np.array(Image.open(pad_img_dir / pad_df.iloc[0]['img_id']).convert('RGB'))

    fig, axes = plt.subplots(2, 6, figsize=(24, 8))

    # Row 1: Dermoscopic image + augmentations
    axes[0,0].imshow(derm_img); axes[0,0].set_title('Original\n(dermoscopic)')
    augs_derm = [
        ('Vignette Remove', DermoscopeVignetteRemove(p=1.0)),
        ('Color Cast', DermoscopeColorCast(p=1.0)),
        ('Hair Overlay', HairOverlay(p=1.0)),
        ('Gel Bubbles', DermoscopeGelBubbles(p=1.0)),
        ('All Combined', None),
    ]
    np.random.seed(42)
    for i, (name, aug) in enumerate(augs_derm):
        if name == 'All Combined':
            result = get_dermoscope_removal_pipeline(p=1.0)(image=derm_img)['image']
        else:
            result = aug(image=derm_img)['image']
        axes[0, i+1].imshow(result)
        axes[0, i+1].set_title(name)

    # Row 2: Smartphone image + augmentations
    axes[1,0].imshow(phone_img); axes[1,0].set_title('Original\n(smartphone)')
    augs_phone = [
        ('Vignette Add', DermoscopeVignetteAdd(p=1.0)),
        ('Color Cast', DermoscopeColorCast(p=1.0)),
        ('Gel Bubbles', DermoscopeGelBubbles(p=1.0)),
        ('Hair Overlay', HairOverlay(p=1.0)),
        ('All Combined', None),
    ]
    for i, (name, aug) in enumerate(augs_phone):
        if name == 'All Combined':
            result = get_dermoscope_addition_pipeline(p=1.0)(image=phone_img)['image']
        else:
            result = aug(image=phone_img)['image']
        axes[1, i+1].imshow(result)
        axes[1, i+1].set_title(name)

    for ax in axes.flat:
        ax.axis('off')

    plt.suptitle('Domain Bridging Augmentations', fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.savefig('../results/augmentation_demo.png', dpi=150, bbox_inches='tight')
    plt.show()

## 9. Taxonomy Options for Triage

Based on the data survey, here are the classification taxonomy options to consider.

### Option A: Binary (Current) — Benign vs Malignant

**Pros**:
- Simplest model, highest statistical power per class
- Directly answers "should I worry about cancer?"
- All four datasets map cleanly to this

**Cons**:
- Can't distinguish "normal mole" from "benign tumor needing monitoring"
- Non-neoplastic conditions (eczema, infections) must be excluded or mislabeled
- No "this isn't even a lesion" output for normal skin photos
- Loses Fitzpatrick17k non-neoplastic data (~48% of dataset)

### Option B: Ternary — Benign / Malignant / Non-Neoplastic

**Pros**:
- Uses ALL of Fitzpatrick17k (recovers ~8k images)
- Non-neoplastic conditions (inflammatory, infectious) get "see a doctor but probably not cancer"
- Maps naturally to triage tiers: green/yellow/red

**Cons**:
- Only Fitzpatrick17k has native three-partition labels
- HAM10000, DDI, PAD don't have non-neoplastic samples
- Still no "normal skin" class

### Option C: Binary + Confidence-Based Triage (Recommended)

**Pros**:
- Keep binary classification (benign/malignant) for maximum training data
- Use model confidence to create triage tiers (low/moderate/high concern)
- Add an out-of-distribution detector for "not a skin lesion" cases
- Fitzpatrick17k non-neoplastic can be included as benign (inflammatory != cancer)

**Cons**:
- Confidence calibration is tricky
- OOD detection adds complexity

### Option D: Binary + Normal Skin Class (Three-Class)

**Pros**:
- Explicitly handles "this is normal skin" — critical for real-world deployment
- Three classes: normal/benign lesion/malignant lesion
- Most clinically useful for screening

**Cons**:
- **Need additional data**: none of our datasets contain "normal skin" images
- Would need a dataset like ISIC with nevi prominently, or synthetic normal skin data
- Risk of class confusion between "normal skin" and "benign mole"

In [ ]:
# Quantify what each option gives us in terms of training data
print('=== Training Data Under Each Taxonomy ===')
print()

# Option A: Binary
n_ham = len(ham_df)
n_ddi = len(ddi_df)
n_fitz_binary = len(fitz_df[fitz_df['three_partition_label'].isin(['benign','malignant'])])
n_pad = len(pad_df)
total_a = n_ham + n_ddi + n_fitz_binary + n_pad
print(f'Option A (Binary): {total_a:,} total')
print(f'  HAM10000={n_ham:,}  DDI={n_ddi}  Fitz17k={n_fitz_binary:,}  PAD={n_pad:,}')
print(f'  Lost: {len(fitz_df) - n_fitz_binary:,} non-neoplastic from Fitz17k')
print()

# Option B: Ternary
total_b = n_ham + n_ddi + len(fitz_df) + n_pad
print(f'Option B (Ternary): {total_b:,} total')
print(f'  Recovers {len(fitz_df) - n_fitz_binary:,} non-neoplastic images')
print(f'  But only Fitz17k has non-neoplastic labels — other datasets have no samples')
print()

# Option C: Binary + OOD
n_fitz_with_nonneo = len(fitz_df)  # include non-neoplastic as benign
total_c = n_ham + n_ddi + n_fitz_with_nonneo + n_pad
print(f'Option C (Binary + confidence triage): {total_c:,} total')
print(f'  Includes non-neoplastic as benign (not cancer = benign for triage)')
print(f'  Needs: confidence calibration, OOD detection for non-lesion images')
print()

# Option D: Three-class with normal
print(f'Option D (Binary + Normal Skin): {total_a:,} + normal skin dataset')
print(f'  Needs: additional dataset of normal skin / healthy moles')
print(f'  Candidates: ISIC Archive (large nevi set), web-scraped normal skin')
print(f'  Note: HAM10000 nv class (n={ham_counts.get("nv",0):,}) is effectively "normal moles"')

## 10. Detailed Condition Breakdown: What's Actually in Each Class?

Before deciding on taxonomy, let's see what specific conditions live in each dataset.

In [ ]:
# Fitzpatrick17k detailed condition breakdown
print('=== Fitzpatrick17k: Top Conditions by Partition ===')
for partition in ['benign', 'malignant', 'non-neoplastic']:
    subset = fitz_df[fitz_df['three_partition_label'] == partition]
    print(f'\n{partition.upper()} ({len(subset)} images):')
    top = subset['label'].value_counts().head(10)
    for cond, count in top.items():
        print(f'  {cond:40s} n={count}')

In [ ]:
# DDI detailed disease breakdown
print('=== DDI: All Diseases ===')
for (disease, mal), group in ddi_df.groupby(['disease', 'malignant']):
    tones = group['skin_tone'].value_counts().to_dict()
    print(f'  {"MAL" if mal else "BEN"} {disease:35s} n={len(group):3d}  tones={tones}')

In [ ]:
# Cross-dataset label overlap analysis
print('=== Cross-Dataset Label Overlap ===')
print()
print('Conditions present in multiple datasets:')
print()

# Rough mapping of equivalent conditions across datasets
cross_conditions = {
    'Melanoma': {
        'HAM10000': ('mel', ham_df['dx'].eq('mel').sum()),
        'DDI': ('melanoma*', ddi_df['disease'].str.contains('melanoma', case=False, na=False).sum()),
        'Fitz17k': ('melanoma', fitz_df['label'].str.contains('melanoma', case=False, na=False).sum()),
        'PAD': ('MEL', (pad_df['diagnostic']=='MEL').sum()),
    },
    'Basal Cell Carcinoma': {
        'HAM10000': ('bcc', ham_df['dx'].eq('bcc').sum()),
        'DDI': ('bcc*', ddi_df['disease'].str.contains('basal', case=False, na=False).sum()),
        'Fitz17k': ('bcc', fitz_df['label'].str.contains('basal', case=False, na=False).sum()),
        'PAD': ('BCC', (pad_df['diagnostic']=='BCC').sum()),
    },
    'Squamous Cell Carcinoma': {
        'HAM10000': ('akiec(related)', ham_df['dx'].eq('akiec').sum()),
        'DDI': ('scc*', ddi_df['disease'].str.contains('squamous', case=False, na=False).sum()),
        'Fitz17k': ('scc', fitz_df['label'].str.contains('squamous', case=False, na=False).sum()),
        'PAD': ('SCC', (pad_df['diagnostic']=='SCC').sum()),
    },
    'Nevus (Mole)': {
        'HAM10000': ('nv', ham_df['dx'].eq('nv').sum()),
        'DDI': ('nevus*', ddi_df['disease'].str.contains('nevus', case=False, na=False).sum()),
        'Fitz17k': ('nevus', fitz_df['label'].str.contains('nevus', case=False, na=False).sum()),
        'PAD': ('NEV', (pad_df['diagnostic']=='NEV').sum()),
    },
    'Seborrheic Keratosis': {
        'HAM10000': ('bkl(includes)', ham_df['dx'].eq('bkl').sum()),
        'DDI': ('sk*', ddi_df['disease'].str.contains('seborrheic', case=False, na=False).sum()),
        'Fitz17k': ('sk', fitz_df['label'].str.contains('seborrheic', case=False, na=False).sum()),
        'PAD': ('SEK', (pad_df['diagnostic']=='SEK').sum()),
    },
}

print(f'{"Condition":30s} {"HAM10000":>10s} {"DDI":>10s} {"Fitz17k":>10s} {"PAD":>10s}')
print('-' * 75)
for condition, datasets in cross_conditions.items():
    vals = [f'{datasets[ds][1]}' for ds in ['HAM10000', 'DDI', 'Fitz17k', 'PAD']]
    print(f'{condition:30s} {vals[0]:>10s} {vals[1]:>10s} {vals[2]:>10s} {vals[3]:>10s}')

## 11. Augmentation Strategy Assessment

### Currently Implemented

| Strategy | Code Location | Purpose |
|---|---|---|
| Domain bridging (vignette add/remove) | `dermoscope_aug.py` | Break dermoscope-pathology correlation |
| Gel bubble artifacts | `dermoscope_aug.py` | Simulate dermoscope contact fluid |
| Polarized color cast | `dermoscope_aug.py` | Simulate dermoscope lighting |
| Hair overlay | `dermoscope_aug.py` | Simulate hair occlusion |
| Lighting variation | `augmentations.py` | Robustness to exam room lighting |
| Sensor noise | `augmentations.py` | Robustness to camera quality |
| Compression artifacts | `augmentations.py` | Robustness to image uploads |
| Geometric (flip, rotate) | `augmentations.py` | Standard invariances |

### Not Yet Implemented (State of the Art)

| Strategy | Rationale | Priority |
|---|---|---|
| **Color constancy / white balance** | Different cameras produce different color casts; standardizing improves cross-device generalization | HIGH |
| **CutMix / MixUp** | Regularization that helps with small datasets; proven on ISIC challenges | MEDIUM |
| **Skin tone-aware color jitter** | Standard color jitter may push light skin darker or vice versa unnaturally | MEDIUM |
| **Ruler/marker artifact overlay** | Clinical photos often have rulers, ink marks; model should ignore these | LOW |
| **Background randomization** | Dermoscopic images have dark backgrounds; phone photos have variable backgrounds | MEDIUM |
| **Diffusion-based synthetic data** | Generate synthetic lesion images for underrepresented classes (esp. melanoma on dark skin) | LOW (complexity) |

In [ ]:
# Analyze which augmentations would help most based on domain gaps
print('=== Domain Gap Summary ===')
print()
for dataset in stats_df['dataset'].unique():
    s = stats_df[stats_df['dataset'] == dataset]
    print(f'{dataset}:')
    print(f'  Resolution: {s["width"].mean():.0f}x{s["height"].mean():.0f} '
          f'(range: {s["width"].min():.0f}-{s["width"].max():.0f})')
    print(f'  Brightness: {s["brightness"].mean():.1f} +/- {s["brightness"].std():.1f}')
    print(f'  Contrast (std): {s["std"].mean():.1f} +/- {s["std"].std():.1f}')
    print(f'  Color (R/G/B): {s["mean_r"].mean():.0f}/{s["mean_g"].mean():.0f}/{s["mean_b"].mean():.0f}')
    print()

## 12. Additional Dataset Candidates

### For Normal Skin / Healthy Moles

| Dataset | Images | Contains Normal? | Access |
|---|---|---|---|
| **ISIC 2019/2020 Archive** | 25,000+ | Yes — large "nevus" (NV) class = normal moles | Public via ISIC API |
| **BCN20000** | 19,424 | Yes — NV is largest class (~30%) | Via ISIC Archive |
| **Derm7pt** | 1,011 | Dermoscopic + clinical pairs, includes nevi | Academic request |
| **SD-198/SD-260** | 6,584 | General dermatology, some normal reference images | Academic |

### Assessment

**Do we actually need a "normal skin" class?**

For a **triage app** (not a diagnostic tool), the answer is nuanced:

1. **If input is always a lesion photo**: Binary is sufficient. User pre-selects the lesion.
2. **If input is any skin photo**: Need normal class or OOD detection.
3. **Our approach (PLAN.md)**: Users upload a photo of a specific lesion they're concerned about.
   This means the input is pre-selected — users won't upload photos of their healthy forearm.

**Recommendation**: Keep binary, but:
- The large HAM10000 `nv` class (6,705 images) already serves as "normal mole" representation
- Include Fitzpatrick17k non-neoplastic as benign (not cancer) to increase data volume
- Add confidence-based triage tiers (already in our triage.py)
- Consider adding ISIC 2019 NV data for more domain diversity in the "benign mole" class

In [ ]:
# Summary statistics for the recommended approach
print('=== RECOMMENDED APPROACH: Option C (Binary + Non-Neoplastic as Benign) ===')
print()

# Recount with non-neoplastic included as benign
benign_total = (
    ham_labels.eq(0).sum() +  # HAM benign
    ddi_labels.eq(0).sum() +  # DDI benign
    (fitz_df['three_partition_label'].isin(['benign', 'non-neoplastic'])).sum() +  # Fitz benign + non-neo
    pad_labels.eq(0).sum()    # PAD benign
)

malignant_total = (
    ham_labels.eq(1).sum() +  # HAM malignant
    ddi_labels.eq(1).sum() +  # DDI malignant
    (fitz_df['three_partition_label'] == 'malignant').sum() +  # Fitz malignant
    pad_labels.eq(1).sum()    # PAD malignant
)

total = benign_total + malignant_total
print(f'Total samples: {total:,}')
print(f'  Benign:    {benign_total:,} ({100*benign_total/total:.1f}%)')
print(f'  Malignant: {malignant_total:,} ({100*malignant_total/total:.1f}%)')
print(f'  Ratio:     {benign_total/malignant_total:.1f}:1 (benign:malignant)')
print()
print('Advantages:')
print(f'  - Recovers {(fitz_df["three_partition_label"]=="non-neoplastic").sum():,} additional images from Fitz17k')
print(f'  - Total goes from {total_a:,} to {total:,} ({100*(total-total_a)/total_a:.0f}% more data)')
print(f'  - Non-neoplastic conditions are "not cancer" which aligns with benign for triage')
print(f'  - Confidence-based triage tiers handle ambiguity without extra classes')

## 13. Summary & Decision Points

### Findings

1. **Class imbalance is severe** across all datasets (benign >> malignant). Our weighted sampling approach is essential.

2. **Fitzpatrick coverage is poor for dark skin** (types V-VI) in all datasets except DDI. PAD-UFES-20 has only 11 samples with FST V-VI.

3. **Domain gap is real and measurable** — brightness, resolution, and color distributions differ significantly between dermoscopic, clinical, and smartphone images.

4. **Non-neoplastic exclusion loses ~48% of Fitzpatrick17k** — including these as benign recovers significant data volume.

5. **No "normal skin" class is needed** for the current triage use case (user-selected lesion photos), but confidence-based triage tiers serve a similar purpose.

### Recommended Decisions

| Decision | Recommendation | Rationale |
|---|---|---|
| **Label taxonomy** | Binary (benign/malignant) | Maximum training data, all datasets align |
| **Non-neoplastic** | Include as benign | Not cancer = benign for triage purposes |
| **Normal skin class** | Not needed now | Users pre-select lesions; OOD detection handles edge cases |
| **Additional dataset** | Consider ISIC 2019 NV for more benign mole diversity | Optional enrichment |
| **Augmentation priority** | Add color constancy / white balance normalization | Biggest remaining domain gap |
| **Confidence triage** | Already implemented in triage.py | Low/moderate/high concern tiers |